In [ ]:
import cv2

previous = 0

for i in range (300 ,1100):
    I = cv2.imread('input/in%06d.jpg' % i)
    I_gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    I_gray = I_gray.astype('uint8')
    current = I_gray
    frame = cv2.absdiff(current, previous)

    T, thresh = cv2.threshold(frame,15,255,cv2.THRESH_BINARY)
    #B = 1*(I > 10)

    cv2.imshow("thresh",thresh)
    cv2.waitKey (10)
    previous = current
cv2.destroyAllWindows()

# D -- input array
# 10 -- threshold value
# 255 -- maximum value to use with the THRESH_BINARY and THRESH_BINARY_INV thresholding types
# cv2.THRESH_BINARY -- thresholding type
# T - our threshold value
# thresh - output image